In [1]:
import torch
from pathlib import Path

from app.model.modules.utils.checkpointer import Checkpointer
from app.model.modules.utils.tokenizer import Tokenizer

checkpoint_path = Path('app/data/models/11/state.pkl')
model, optimizer, evaluator, config = Checkpointer.load_checkpoint(
    checkpoint_path)
tokenizer = Tokenizer()
tokenizer.load()

Loaded checkpoint app/data/models/11/state.pkl


In [2]:
sequence = 'It takes practice, Lily'
max_gen_len = 200
end_token = '<|endoftext|>'
while max_gen_len:
    tokens = torch.tensor(tokenizer.encode(sequence))
    next_token = model.generate_next_token(tokens).item()
    decoded_next_token = tokenizer.decode([int(next_token)])
    if sequence.endswith(end_token):
        break
    sequence += decoded_next_token
    max_gen_len -= 1
sequence

'It takes practice, Lily the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p the p'

In [6]:
evaluator.metrics['cross_entropy'].values

[tensor(8.5171),
 tensor(8.5173),
 tensor(8.5118),
 tensor(8.4971),
 tensor(8.4740),
 tensor(8.4422),
 tensor(8.3967),
 tensor(8.3660),
 tensor(8.2679),
 tensor(7.9931),
 tensor(7.7598)]